In [1]:
import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=gpu'
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin/"
#import sys
#sys.path.append('/data/fs4/home/bradh/')
import theano
import numpy as np
import random
from copy import copy

import learningfunctions

import blocks
from blocks.bricks import Linear, Softmax, Softplus, NDimensionalSoftmax, BatchNormalizedMLP, \
                                Rectifier, Logistic, Tanh, MLP
from blocks.bricks.recurrent import GatedRecurrent, LSTM
from blocks.bricks.parallel import Fork
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform
from blocks.bricks.cost import BinaryCrossEntropy, CategoricalCrossEntropy
from blocks.filter import VariableFilter
from blocks.roles import PARAMETER
from blocks.graph import ComputationGraph

import theano
from theano import tensor as T

Using gpu device 0: TITAN X (Pascal) (CNMeM is disabled, CuDNN 5105)
/opt/conda/envs/altair/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:597: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)


In [2]:
data = ['this is data', 'we like data', 'once upon a time']

In [ ]:
raw_data = T.tensor4("raw_data")
rnn_type = "gru" # "lstm"
orig_dims = 70
compressed_dims = 20
weight_stdev = 0.2
rnn_bias_init = Constant(0.0)
rnn_weight_init = IsotropicGaussian(weight_stdev)
line_weight_init = IsotropicGaussian(weight_stdev)
line_bias = Constant(1.0)
learning_rate = 0.0001
clip_threshold = 1

# TODO add learning rate decay over time

In [3]:
X = T.tensor4('X')
rnnType = 'gru'
dimIn = 70
dim = 20
wtstd = 0.2
rnnbias_init = Constant(0.0)
rnnwt_init = IsotropicGaussian(wtstd)
linewt_init = IsotropicGaussian(wtstd)
line_bias = Constant(1.0)
learning_rate = 0.0001
clippings = 1
#ADD lr decay

In [4]:
if rnnType == 'gru':
    rnn = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

fork = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimIn, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)



In [5]:
def onestepEnc(X):
    data1, data2 = fork.apply(X) 

    if rnnType == 'gru':
        hEnc = rnn.apply(data1, data2) 
    else:
        hEnc, _ = rnn.apply(data2)

    return hEnc

hEnc, _ = theano.scan(onestepEnc, X) 


In [6]:
fork.initialize()
rnn.initialize()

layer1Fun = theano.function([X], hEnc, allow_input_downcast=True)

In [7]:
fakeData = np.random.rand(3,50,1,70).astype('float32')

In [8]:
if rnnType == 'gru':
    rnn2 = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn2 = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

fork2 = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)



def onestepEnc2(hEnc):
    data3, data4 = fork2.apply(hEnc) 

    if rnnType == 'gru':
        hEnc2 = rnn2.apply(data3, data4) 
    else:
        hEnc2, _ = rnn2.apply(data4)

    return hEnc2, data3

[hEnc2, data3], _ = theano.scan(onestepEnc2, hEnc) 

fork2.initialize()
rnn2.initialize()

layer2Fun = theano.function([X], hEnc2, allow_input_downcast=True)

In [9]:
if rnnType == 'gru':
    rnn3 = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn3 = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

fork3 = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

forkD = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dim, output_dims=[dimIn, dimIn * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

def onestepEnc3(hEnc2):
    data5, data6 = fork3.apply(hEnc2) 

    if rnnType == 'gru':
        hEnc3 = rnn3.apply(data5, data6) 
    else:
        hEnc3, _ = rnn3.apply(data6)

    return hEnc3

hEnc3, _ = theano.scan(onestepEnc3, hEnc2) 
h4decoder = hEnc3[:,-1,:,:].reshape((-1, 1,1,20))

h4reshape, _ = forkD.apply(h4decoder)

forkD.initialize()
fork3.initialize()
rnn3.initialize()

layer3Fun = theano.function([X], h4reshape, allow_input_downcast=True)

In [10]:
if rnnType == 'gru':
    rnn4 = GatedRecurrent(dim=dimIn, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn4 = LSTM(dim=dimIn, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

#TOTHINK: transform before the decoder or after
fork4 = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimIn, output_dims=[dimIn, dimIn * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

targets = T.concatenate((h4reshape, X[:,:-1, :,:]), axis=1)

def onestepEnc4(targets):
    data7, data8 = fork4.apply(targets) 

    if rnnType == 'gru':
        hDec = rnn4.apply(data7, data8) 
    else:
        hDec, _ = rnn4.apply(data8)

    return hDec

hDec, _ = theano.scan(onestepEnc4, targets) 

fork4.initialize()
rnn4.initialize()

layer4Fun = theano.function([X], hDec, allow_input_downcast=True)

decFun = theano.function([X], targets, allow_input_downcast=True)

In [11]:
predTargets = T.exp(hDec)/T.sum(T.exp(hDec), axis=(3,2), keepdims=True)
#precost = -X.squeeze*T.log(predTargets.squeeze()) - (1-X.squeeze())*T.log(1-predTargets.squeeze())

#ADDLATER: beam search
cost = T.mean(T.sum(T.nnet.categorical_crossentropy(predTargets, X), axis = 1))

In [12]:
cg = ComputationGraph([cost])
params = VariableFilter(roles = [PARAMETER])(cg.variables)

###To check gradients for explosion/shrinkage
#gradients = T.grad(costClass, paramsClass)
#gradients = clip_norms(gradients, clippings)
#gradientFun = theano.function([X,Y], gradients, allow_input_downcast=True)

learning = learningfunctions.Learning(cost,params,learning_rate,l1=0.,l2=0.,maxnorm=0.,c=clippings)
updates = learning.Adam() 

print('compiling graph you talented soul')
classifierTrain = theano.function([X], [cost, predTargets], 
                                  updates=updates, allow_input_downcast=True)
#classifierPredict = theano.function([X], [softoutClass, attEncpred, attContextpred], allow_input_downcast=True)
classifierPredict = theano.function([X], predTargets, allow_input_downcast=True)

compiling graph you talented soul


In [ ]:
with open('test.txt', 'r') as f:
    charDict2 = f.readlines()[]

In [ ]:
charDict2 = charDict2[0]

In [ ]:
charDict2

In [ ]:
'\\' in charDict2

In [ ]:
r'234\55'[3] in charDict2